Faites une analyse exploratoire des données (valeurs manquantes, nombre d'exemples positifs,..)

In [6]:
#charger les biblithèques numpy et pandas
import pandas as pd
import numpy as np

Avec Sklearn, utilisez l'algorithme Kppv  pour la classification des données

In [7]:
#lire la base de données à partir du fichier csv diabetes
dataset=pd.read_csv('diabetes.csv')
dataset.info()

FileNotFoundError: [Errno 2] No such file or directory: 'diabetes.csv'

In [ ]:
#Afficher les statistiques de la base en utilisant la fonction describe()
dataset.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [ ]:
#Combien de valeur null par colonne?
dataset.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [ ]:
dataset.head(3)

NameError: name 'dataset' is not defined

In [ ]:
X = dataset.iloc[:,:-1] #Récupérer toutes les lignes et toutes les colonnes sauf la dernière colonne de la base
y = dataset['Outcome'] #Récupérer la colonne outcome qui représente le target et stocker la dans la vaiable y

Utilisez 60% de la donnée pour l'apprentissage et les 40% restants pour le test, avec un k=3. Quels précision et rappel optenez-vous? Dessinez le tableau de contingence.

In [ ]:
#Diviser la base de donnée en deux sous-ensembles test et train avec une taille de 40%, 60%, respectivement
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, stratify=y)

ModuleNotFoundError: No module named 'sklearn'

Optimisez les paramètres avec la validation croisée, choisissez la meilleure valeur du paramètre K  qui donne les meilleurs scores précision et rappel.  L'impact de la variation de la valeur de K peut être étudié en utilisant l'intervalle [3 -11].

In [ ]:
#Charger le grid search qui cherche les paramètres optimaux d'un modèle
from sklearn.model_selection import  GridSearchCV
#Charger le classificateur KNN
from sklearn.neighbors import KNeighborsClassifier
grid_params={
    'n_neighbors':[3,5,7,9,11],
    'metric':['euclidean','manhattan']
}

#Créer un objet grid search avec les paramètres et le modèle à tester
gs=GridSearchCV(
    KNeighborsClassifier(),
    grid_params,
    cv=3 #nombre de folds dans la cross validation
)
gs_results=gs.fit(X_train, y_train) #lancer le grid search et la cross validation

In [ ]:
gs_results.best_params_ #Afficher les meilleurs paramètres trouvés

{'metric': 'manhattan', 'n_neighbors': 9}

Le classificateur donne t-il de bons résultats?

In [ ]:
#Créer une instance knn avec les paramètres optimaux trouvés
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier(n_neighbors=9,metric='manhattan')
#Lancer l'apprentissage
knn.fit(X_train, y_train)#entraîner le modèles
y_pred=knn.predict(X_test) #evaluer le modèles

In [ ]:
#afficher la précision et le rappel
from sklearn.metrics import precision_score, recall_score
print('precision=',precision_score(y_test,y_pred))
print('recall=',recall_score(y_test,y_pred))

precision= 0.6705882352941176
recall= 0.5327102803738317


In [ ]:
#Afficher la matrice de confusion
from sklearn.metrics import  confusion_matrix
confusion_matrix(y_test, y_pred)

array([[173,  28],
       [ 50,  57]], dtype=int64)

Si vous utilisez 70% de la donnée pour l'apprentissage et le K obtenu dans la question , les résultats changent-ils? Comment  assurer la stabilité des performances du  modèle? 

In [ ]:
#Vérifier la  statbilité du modèle avec 30% des données pour le test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify=y)
#entrainer une nouvelle instance knn avec la nouvelle décomposition de données
knn= KNeighborsClassifier(n_neighbors=9,metric='manhattan', algorithm='brute')
#Lancer l'entrainement
knn.fit(X_train, y_train)
#Evaluer le modèle sur l'ensemble de test
y_pred=knn.predict(X_test)

In [ ]:
#Afficher la matrice de confusion
(VP,FN, FP,VN)=confusion_matrix(y_test, y_pred).ravel()
(VP,FN, FP,VN)

(173, 28, 50, 57)

Pour assuer la statbilité des tests, nous allons utiliser la validation croisée. Ci-aprés nous présentons deux classes sklearn qui implémente la validation croisée

In [11]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
yy=np.arange(768)
XX=X_train.to_numpy() # marche uniquement avec la version r version is less than 0.24.1: pip install --upgrade  to_numpy() is a new method
for train_index, test_index in kf.split(yy):
    X_train2, X_test2 = XX[train_index], XX[test_index]
    y_train2, y_test2 = y[train_index], y[test_index]
    knn.fit(X_train2, y_train2)
    y_pred=knn.predict(X_test2)
    print(precision_score(y_test2,y_pred))

NameError: name 'X_train' is not defined

In [ ]:
#Appliquer une cross validation avec le knn ayant les paramètres optimaux
from sklearn.model_selection import cross_val_score
#create a new KNN model
knn_cv = KNeighborsClassifier(n_neighbors=9, metric='manhattan')#train model with cv of 5 
cv_scores = cross_val_score(knn_cv, X, y, cv=5)#print each cv score (accuracy) and average them
print(cv_scores)
print('cv_scores mean:{}'.format(np.mean(cv_scores)))

[ 0.75324675  0.70779221  0.74025974  0.76470588  0.73856209]
cv_scores mean:0.7409133350309821
